In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#project
from macroecon_wrappy import (
    Auth,
    Metric,
    Measure
)
from macroecon_wrappy.adapters import (
    FredApi,
    YahooFin
)

#external
from fredapi import Fred
import yfinance as yf

#system
from pathlib import Path

In [3]:
#fred
secrets_path = Path('/workspaces/API-macroecon-wrappy/SECRETS.yaml')
cache_path = Path('/workspaces/API-macroecon-wrappy/tests/tmp/')
auth = Auth(secrets_path)
auth.load_secrets()
fred = Fred(api_key=auth.data['API_KEY_FED'])
FredApi.set_wrapper(fred)
gdp = FredApi.get_data('GDP')
gdi = FredApi.get_data('GDI')

In [10]:
#yahoo
YahooFin.set_wrapper(yf)
msft = YahooFin.get_data(tickers='MSFT')

[*********************100%%**********************]  1 of 1 completed


In [16]:
output = Measure([gdp, gdi, msft])
df = output.df()

In [17]:
df.head()

,GDP,GDI,MSFT
2024-07-05,NaN,NaN,463.660004
2024-07-03,NaN,NaN,459.449997
2024-07-02,NaN,NaN,456.349991
2024-07-01,NaN,NaN,451.514999
2024-06-28,NaN,NaN,450.895004
